In [104]:
import nltk
import spacy
import string
import gensim
import asent
import stylecloud

from gensim import corpora
from pprint import pprint
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from spacy.lang.da.stop_words import STOP_WORDS
from spacy.lang.da.examples import sentences 
from nltk.stem.snowball import DanishStemmer

In [72]:
#conda install -c conda-forge spacy-model-da_core_news_sm
# nltk.download('stopwords')
# nltk.download('wordnet')
# nltk.download('omw-1.4')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('averaged_perceptron_tagger')

### First example i have choosen  KMD and and article about them.

#### Starting with loading in the data

In [106]:
#with open('kmd.txt', 'r') as f:
#    text = f.read()

In [77]:
with open('kmdAftale.txt', 'r') as f:
    text = f.read()

In [78]:
text

'Region Sjælland har netop gennemført et større udbud om fremtidig levering af regionens løn- og vagtplansystemer. KMD har genvundet udbuddet, og det betyder, at den danske it-koncern fra 2024-2032 fortsat skal levere it-løsningerne KMD Opus Personale og KMD Opus Vagtplan til Region Sjælland.\n\n”Vi har haft en omfattende og grundig udbudsproces med mange aktører. Vores succeskriterier har været rammesættende for udbuddet - vi vil have et standardsystem som kan støtte og lette personaleopgaverne for vores ledere og en løsning hvor både løn og vagtplan er fra samme leverandør. Valget faldt på KMD, der er vores nuværende leverandør, fordi vi her fik en sammenhængende og moderne it-løsning med systemer i symbiose. Derfor vil der både være stor genkendelighed for medarbejderne, men også fornyelse på flere områder, hvor vi ser frem til at udnytte de mange nye muligheder i vagtplanssystemet og til at øge de digitale arbejdsprocesser inden for HR-området,” siger Marianne Evers, HR-direktør i 

# Tokenize the text

In [79]:
tokens = word_tokenize(text)

In [80]:
#tokens

# Remove Stop Words in "Danish"

In [81]:
stop_words = set(stopwords.words('danish'))
filtered_tokens = [token for token in tokens if token.lower() not in stop_words]

In [82]:
filtered_tokens

['Region',
 'Sjælland',
 'netop',
 'gennemført',
 'større',
 'udbud',
 'fremtidig',
 'levering',
 'regionens',
 'løn-',
 'vagtplansystemer',
 '.',
 'KMD',
 'genvundet',
 'udbuddet',
 ',',
 'betyder',
 ',',
 'danske',
 'it-koncern',
 '2024-2032',
 'fortsat',
 'levere',
 'it-løsningerne',
 'KMD',
 'Opus',
 'Personale',
 'KMD',
 'Opus',
 'Vagtplan',
 'Region',
 'Sjælland',
 '.',
 '”',
 'haft',
 'omfattende',
 'grundig',
 'udbudsproces',
 'aktører',
 '.',
 'Vores',
 'succeskriterier',
 'rammesættende',
 'udbuddet',
 '-',
 'standardsystem',
 'kan',
 'støtte',
 'lette',
 'personaleopgaverne',
 'vores',
 'ledere',
 'løsning',
 'både',
 'løn',
 'vagtplan',
 'samme',
 'leverandør',
 '.',
 'Valget',
 'faldt',
 'KMD',
 ',',
 'vores',
 'nuværende',
 'leverandør',
 ',',
 'fordi',
 'fik',
 'sammenhængende',
 'moderne',
 'it-løsning',
 'systemer',
 'symbiose',
 '.',
 'Derfor',
 'både',
 'stor',
 'genkendelighed',
 'medarbejderne',
 ',',
 'fornyelse',
 'flere',
 'områder',
 ',',
 'ser',
 'frem',
 'udn

# Lemmatize the tokens

#### So that i can reduce the words to their root form, where i use WordNetLemmatizer

In [83]:
lemmatizer = WordNetLemmatizer()
lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]

# Part of Speech Tagging

#### Trying to implement POS tagging which could be helpful to identify the role of each word in the text

#### NNP	proper noun, singular - VBD verb past tense - CD cardinal digit - NN noun, singular (cat, tree and so on) - VB verb (ask) - JJ - This NLTK POS Tag is an adjective (large)
#### the rest can be found here: https://www.guru99.com/pos-tagging-chunking-nltk.html

In [84]:
pos_tagged_tokens = nltk.pos_tag(lemmatized_tokens)

In [85]:
pos_tagged_tokens

[('Region', 'NNP'),
 ('Sjælland', 'NNP'),
 ('netop', 'MD'),
 ('gennemført', 'VB'),
 ('større', 'JJ'),
 ('udbud', 'JJ'),
 ('fremtidig', 'NN'),
 ('levering', 'VBG'),
 ('regionens', 'NNS'),
 ('løn-', 'JJ'),
 ('vagtplansystemer', 'NN'),
 ('.', '.'),
 ('KMD', 'NNP'),
 ('genvundet', 'NN'),
 ('udbuddet', 'NN'),
 (',', ','),
 ('betyder', 'NN'),
 (',', ','),
 ('danske', 'JJ'),
 ('it-koncern', 'JJ'),
 ('2024-2032', 'JJ'),
 ('fortsat', 'NN'),
 ('levere', 'EX'),
 ('it-løsningerne', 'JJ'),
 ('KMD', 'NNP'),
 ('Opus', 'NNP'),
 ('Personale', 'NNP'),
 ('KMD', 'NNP'),
 ('Opus', 'NNP'),
 ('Vagtplan', 'NNP'),
 ('Region', 'NNP'),
 ('Sjælland', 'NNP'),
 ('.', '.'),
 ('”', 'NN'),
 ('haft', 'NN'),
 ('omfattende', 'NN'),
 ('grundig', 'NN'),
 ('udbudsproces', 'NNS'),
 ('aktører', 'NN'),
 ('.', '.'),
 ('Vores', 'VBZ'),
 ('succeskriterier', 'JJR'),
 ('rammesættende', 'NN'),
 ('udbuddet', 'SYM'),
 ('-', ':'),
 ('standardsystem', 'NN'),
 ('kan', 'JJ'),
 ('støtte', 'NN'),
 ('lette', 'NN'),
 ('personaleopgaverne', 'J

### Named Entity Recognition where im trying ti identify entities in the text

### this have been trained to recognize four types of entities: location (LOC), organizations (ORG), person (PER) and Miscellaneous (MISC).

In [86]:
nlp = spacy.load('da_core_news_sm')
doc = nlp(text)
entities = [(entity.text, entity.label_) for entity in doc.ents]

In [87]:
entities

[('KMD', 'ORG'),
 ('danske', 'MISC'),
 ('it-løsningerne', 'MISC'),
 ('KMD Opus Personale', 'ORG'),
 ('KMD Opus Vagtplan', 'ORG'),
 ('Region Sjælland', 'LOC'),
 ('KMD', 'ORG'),
 ('HR-området', 'MISC'),
 ('Marianne Evers', 'PER'),
 ('Region Sjælland', 'LOC'),
 ('KMD', 'ORG'),
 ('Region Sjælland', 'ORG'),
 ('Tom Johan Jensen', 'PER'),
 ('Human Capital Management', 'PER'),
 ('KMD', 'ORG'),
 ('Netop', 'LOC'),
 ('Jørgen Munk Nielsen', 'PER'),
 ('Vagtplan', 'PER'),
 ('KMD', 'ORG'),
 ('Region Sjælland', 'PER'),
 ('Region Sjællands', 'PER'),
 ('Region Sjælland', 'PER'),
 ('Business Intelligence-værktøj', 'PER'),
 ('SAP-platform', 'MISC'),
 ('KMD', 'ORG'),
 ('SAP-certificeringer', 'MISC'),
 ('Global', 'ORG'),
 ('Expertise', 'ORG'),
 ('KMD', 'ORG'),
 ('s', 'ORG'),
 ('KMD Opus Personale', 'ORG'),
 ('KMD Opus Vagtplan', 'ORG'),
 ('KMD', 'ORG'),
 ('KMD', 'ORG'),
 ('danske', 'MISC'),
 ('Danmark', 'LOC'),
 ('Skandinavien', 'LOC'),
 ('KMD-koncernen', 'MISC'),
 ('Danmark', 'LOC'),
 ('Norge', 'LOC'),
 ('

In [91]:
noun_chunks = [chunk.text for chunk in doc.noun_chunks]
print(noun_chunks)
#sentences = [sent.text for sent in doc.sents]
#print(sentences)

['Region Sjælland', 'et større udbud', 'fremtidig levering', 'regionens', 'vagtplansystemer', 'KMD', 'udbuddet', 'det', 'den danske it-koncern fra 2024-2032', 'it-løsningerne', 'KMD', 'Opus', 'Personale', 'KMD', 'Opus Vagtplan', 'Region Sjælland', 'Vi', 'en omfattende og grundig udbudsproces', 'mange aktører', 'Vores succeskriterier', 'udbuddet', 'vi', 'et standardsystem', 'personaleopgaverne', 'vores ledere', 'en løsning', 'løn', 'vagtplan', 'samme leverandør', 'Valget', 'KMD', 'vores nuværende leverandør', 'vi', 'en sammenhængende og moderne it-løsning', 'systemer', 'symbiose', 'stor genkendelighed', 'medarbejderne', 'fornyelse', 'flere områder', 'vi', 'de mange nye muligheder', 'vagtplanssystemet', 'HR-området', 'Marianne Evers', 'HR-direktør', 'Region Sjælland', 'KMD', 'man', 'det fortsatte samarbejde', 'Vi', 'strategisk samarbejdspartner', 'Region Sjælland', 'vi', 'et fortsat gensidigt godt samarbejde', 'aftalen', 'vi', 'de kvalitetssystemer', 'regionens', 'medarbejdere', 'vi', 'n

### Making a function that is able to do everything to pre process the text, and then just return the text

In [95]:
# Loading Danish language model in spaCy
nlp = spacy.load('da_core_news_sm')
stemmer = DanishStemmer()

def preprocess(doc):
    # Tokenizinfdg the  text and removing punctuation
    tokens = [token.text for token in nlp(doc) if not token.is_punct]
    
    # Remove alle the stop words, lemmatize tokens, and stem tokens
    tokens = [stemmer.stem(token) for token in tokens if token.lower() not in STOP_WORDS]
    
    # Tag tokens with parts of speech
    pos_tags = [token.pos_ for token in nlp(' '.join(tokens))]
    
    # Only keep nouns and adjectives
    tokens = [token for i, token in enumerate(tokens) if pos_tags[i] in ['NOUN', 'ADJ']]
    
    # Join the tokens into a single string
    filtered_text = ' '.join(tokens)
    return filtered_text

#document = 'Vindmølleproducenten skifter sine 6.000 danske ansatte til den cloudbaserede KMD Payroll Cloud.\n”KMD Payroll Cloud passer perfekt til os, da løsningen optimerer en række processer, og samtidig vil KMD håndtere mange administrationsopgaver. Dermed får vi en delvist outsourcet løsning, hvor vi stadig selv er i kontrol med vores egne data,” siger Kirsten Therkildsen, som er Head of HR Services i Siemens Gamesa.\n”Nu glæder vi os bare til at komme i gang med projektet, og vi ser meget frem til samarbejdet med KMD," tilføjer hun.\nOgså i KMD er der glæde over aftalen. \n”Vi er stolte af, at Siemens Gamesa valgte os som deres partner. Samtidig får vi nu en ny kunde ind, hvor vi integrerer KMD Payroll Cloud med virksomhedens HR-system – Workday – så vi dækker efterhånden integrationen til de førende globale HR-systemer. Det giver nogle gode markedsmæssige muligheder,” siger Kristian de Linde, Business Line Director i KMD.\n”I hele forløbet har vi haft en meget tæt kontakt til kunden – vi har lyttet, forstået kundens behov og i sidste ende tilbudt en løsning, som passer til Siemens Gamesa,” siger Kristian de Linde og sender en stor tak til teamet bag den nye aftale.\nSiemens Gamesa anvender i dag løsningen KMD Nettoløn og er den første af de mange kunder på KMD Nettoløn, som bliver løftet til den cloudbaserede KMD Payroll Cloud.'
document = 'Region Sjælland har netop gennemført et større udbud om fremtidig levering af regionens løn- og vagtplansystemer.', 'KMD har genvundet udbuddet, og det betyder, at den danske it-koncern fra 2024-2032 fortsat skal levere it-løsningerne KMD Opus Personale og KMD Opus Vagtplan til Region Sjælland.\n\n', '”Vi har haft en omfattende og grundig udbudsproces med mange aktører.', 'Vores succeskriterier har været rammesættende for udbuddet - vi vil have et standardsystem som kan støtte og lette personaleopgaverne for vores ledere og en løsning hvor både løn og vagtplan er fra samme leverandør.', 'Valget faldt på KMD, der er vores nuværende leverandør, fordi vi her fik en sammenhængende og moderne it-løsning med systemer i symbiose.', 'Derfor vil der både være stor genkendelighed for medarbejderne, men også fornyelse på flere områder, hvor vi ser frem til at udnytte de mange nye muligheder i vagtplanssystemet og til at øge de digitale arbejdsprocesser inden for HR-området,” siger Marianne Evers, HR-direktør i Region Sjælland.\n\n', 'I KMD ser man meget frem til det fortsatte samarbejde.\n\n', '”Vi er rigtig glade for at blive tilvalgt som strategisk samarbejdspartner af Region Sjælland, og vi ser frem til at bidrage til et fortsat gensidigt godt samarbejde.', 'Med aftalen kommer vi til at levere de kvalitetssystemer, regionens medarbejdere allerede kender godt, og derudover har vi udviklet nye elementer til løsningen,” siger Tom Johan Jensen, direktør for Human Capital Management i KMD.\n\n', 'Masser af fælles udvikling i aftalen\n', 'Netop fokusset på at udvikle nye elementer til den velkendte løsning har både i den forgangne og nye kontraktperiode været et væsentligt element i samarbejdet.\n\n“Et af målene i den nye aftale er at modne fremtidens måde at planlægge arbejdsopgaver på.', 'Det kommer vi til at gøre ved fx at sikre patientfokus gennem øget intelligent automatisering balanceret med en fair planlægning af medarbejdernes arbejdstid, der også afspejler deres individuelle livsfaser og ønsker til nærvær og fravær,” siger Jørgen Munk Nielsen, forretningsdirektør for Vagtplan i KMD og tilføjer:\n\n', '“', 'Vi har i fællesskab intensiveret det gode samarbejde de forgangne år, hvor vi struktureret har set på, hvad der fungerer rigtig godt og hvilke ønsker der er til udvikling.', 'Her er implementeringerne sket i bølger i den takt, Region Sjælland har ønsket det.', 'Det har også betydet, at vi i tæt samarbejde har prioriteret den løbende udvikling af blandt andet funktionalitet og brugeroplevelsen, så indholdet blev nøje afstemt med Region Sjællands behov.', '”\n\nRegion Sjælland får dermed en løsning, der giver sammenhæng og sikkerhed i håndteringen af de mange løn- og personaleopgaver, og som indeholder moderne og fleksible redskaber, der sikrer, at opgaveløsningen sker effektivt og smidigt – blandt andet med et Business Intelligence-værktøj.\n\n', 'Teknologisk er løn- og personaleløsningen bag aftalen bygget på den markedsledende SAP-platform, der er en del af årsagen til, at KMD i 2021 blev udnævnt til et af verdens 10 bedste inden for SAP-certificeringer med Global top 10 Customer Center of Expertise.', 'Vagtplansløsningen er bygget som en smidigt integreret løsning, så data flyder online mellem de to systemer og sikrer, at brugerne altid oplever sammenhæng mellem systemerne og arbejder med de samme opdaterede data.\n\n', 'Kvaliteten af løsningerne og deres mulighed for digital understøttelse af komplekse love, overenskomster og beregninger er resultatet af KMD’s mangeårige erfaring med både teknologien såvel som faglighederne i regionerne.', 'Region Sjælland får med valget af KMD Opus Personale og KMD Opus Vagtplan derfor også adgang til en stor vidensbank bestående af 200 medarbejdere med spidskompetencer inden for løn, overenskomster, arbejdsprocesser og vagtplanlægning.\n\n', 'Om KMD\nKMD er en af de største danske it-virksomheder, der udvikler og leverer software- og serviceløsninger til kommune-, stats- og erhvervssegmentet i Danmark samt udvalgte segmenter i Skandinavien.', 'KMD-koncernen har datterselskaber i Danmark, Norge, Sverige, Finland og Polen.', 'KMD', 'har over 1.500 danske og internationale kunder fra den offentlige og private sektor, hvoraf cirka 800 er danske og udenlandske virksomheder.', 'KMD-koncernen har en årlig omsætning på omkring 4,9 mia. kr. og omkring 3.000 ansatte.', 'KMD er et datterselskab af NEC Corporation, en førende global virksomhed inden for integration af it- og netværksteknologier'
#document is a tuple andi just want it as a string
stringdoc = " ".join(document)
preprocessed_doc = preprocess(stringdoc)
print(preprocessed_doc)

region sjælland stør fremtid levering region løn- vagtplansystem genvund udbud dansk it-koncern it-løsning opus personal kmd opus vagtplan region omfat grund succeskriteri udbud standardsystem led løsning løn vagtplan leverandør valg fald kmd nuvær leverandør sammenhæng it-løsning symbios stor genkend medarbejd forny områd udnyt nye mul vagtplanssystem digital hr-områd hr-direktør region kmd glad tilvalg strategisk samarbejdspartn region bidrag gensid aftal kvalitetssystem region medarbejd kend nye element løsning direktør capital management kmd udvikling aftal fokus nye element velkend løsning båd forgangn nye kontraktperiod mål nye aftal modn fremtid planlæg sikr patientfokus intelligent automatisering balanc fair planlægning medarbejd ønsk nærvær fravær forretningsdirektør vagtplan fællesskab intensiv god samarbejd forgangn år struktur fung ønsk udvikling implementering bølg takt region ønsk betyd samarbejd udvikling indhold nøj afstemt region behov region sjælland løsning sammenhæn

In [96]:
nlp = spacy.blank("da")
nlp.add_pipe("sentencizer")
nlp.add_pipe("asent_da_v1")
doc = nlp(preprocessed_doc)
print(doc._.polarity)

neg=0.0 neu=0.862 pos=0.138 compound=0.9823


### In this example we can again use Asent to obtain more information and in this situation the rated valence of a single token. We can see that both "glad" and "løsning" is rated postive

In [97]:
for token in doc:
    print(f"{token._.polarity} | Valence: {token._.valence} | Negation: {token._.is_negation}")

polarity=0.0 token=region span=region | Valence: 0.0 | Negation: False
polarity=0.0 token=sjælland span=sjælland | Valence: 0.0 | Negation: False
polarity=0.0 token=stør span=stør | Valence: 0.0 | Negation: False
polarity=0.0 token=fremtid span=fremtid | Valence: 0.0 | Negation: False
polarity=0.0 token=levering span=levering | Valence: 0.0 | Negation: False
polarity=0.0 token=region span=region | Valence: 0.0 | Negation: False
polarity=0.0 token=løn- span=løn- | Valence: 0.0 | Negation: False
polarity=0.0 token=vagtplansystem span=vagtplansystem | Valence: 0.0 | Negation: False
polarity=0.0 token=genvund span=genvund | Valence: 0.0 | Negation: False
polarity=0.0 token=udbud span=udbud | Valence: 0.0 | Negation: False
polarity=0.0 token=dansk span=dansk | Valence: 0.0 | Negation: False
polarity=0.0 token=it-koncern span=it-koncern | Valence: 0.0 | Negation: False
polarity=0.0 token=it-løsning span=it-løsning | Valence: 0.0 | Negation: False
polarity=0.0 token=opus span=opus | Valence: 

In [98]:
# visualize model prediction
asent.visualize(doc, style="prediction")

## Im using Latent Dirichlet Allocation trying to identify the main topics or themes discussed in the article. This can give me some ideas what the company is focused on and what they specialize in

### the output that i get is only based on a small text which isn't perfect, and in real-world scenarios, i would need to process a larger text corpus for the LDA model to get better results.

In [99]:
# Tokeniz the text
tokens = gensim.utils.simple_preprocess(preprocessed_doc)

# Creating a dictionary from the tokens
dictionary = corpora.Dictionary([tokens])

# Creating a corpus from the tokens
corpus = [dictionary.doc2bow([token]) for token in tokens]

# Definng the number of topics
num_topics = 1

# Training tthe LDA model
lda_model = gensim.models.LdaModel(corpus=corpus,
                                   id2word=dictionary,
                                   num_topics=num_topics,
                                   random_state=100,
                                   chunksize=100,
                                   passes=10)
pprint(lda_model.print_topics())

[(0,
  '0.027*"dansk" + 0.026*"virksom" + 0.024*"kmd" + 0.024*"datterselskab" + '
  '0.021*"region" + 0.015*"løsning" + 0.015*"koncern" + 0.013*"international" '
  '+ 0.013*"udenlandsk" + 0.013*"udvalg"')]


## This is the output of the LDA model, where each topic is represented as a probability distribution over the words in the corpus. The output shows the top 6 most probable words for each topic, along with the probability for each word.

Topic: 0
Words: 0.167*"token5" + 0.167*"token6" + 0.167*"token3" + 0.167*"token1" + 0.167*"token4" + 0.167*"token2"

This means that the top 6 words for this topic are "token5", "token6", "token3", "token1", "token4", and "token2", and they are equally likely to occur in this topic.

Similarly for the second topic:

Topic: 1
Words: 0.167*"token5" + 0.167*"token6" + 0.167*"token1" + 0.167*"token3" + 0.167*"token4" + 0.167*"token2"

This means that the top 6 words for this topic are "token5", "token6", "token1", "token3", "token4", and "token2", and they are equally likely to occur in this topic.

In [100]:
# Preprocessed text data as an array of Unicode tokens
preprocessed_doc = [["token1", "token2", "token3"], ["token4", "token5", "token6"]]

# Create a dictionary from the preprocessed text data
dictionary = corpora.Dictionary(preprocessed_doc)

# Create a corpus from the preprocessed text data
corpus = [dictionary.doc2bow(text) for text in preprocessed_doc]

# Training LDA model on corpus
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=dictionary,
                                            num_topics=10,
                                            passes=10)

for idx, topic in lda_model.print_topics(num_topics=10, num_words=10):
    print("Topic: {} \nWords: {}".format(idx, topic))

Topic: 0 
Words: 0.167*"token3" + 0.167*"token4" + 0.167*"token6" + 0.167*"token2" + 0.167*"token1" + 0.167*"token5"
Topic: 1 
Words: 0.167*"token3" + 0.167*"token6" + 0.167*"token4" + 0.167*"token2" + 0.167*"token1" + 0.167*"token5"
Topic: 2 
Words: 0.167*"token4" + 0.167*"token6" + 0.167*"token3" + 0.167*"token1" + 0.167*"token2" + 0.167*"token5"
Topic: 3 
Words: 0.167*"token2" + 0.167*"token6" + 0.167*"token4" + 0.167*"token3" + 0.167*"token1" + 0.167*"token5"
Topic: 4 
Words: 0.167*"token3" + 0.167*"token4" + 0.167*"token6" + 0.167*"token2" + 0.167*"token5" + 0.167*"token1"
Topic: 5 
Words: 0.167*"token6" + 0.167*"token1" + 0.167*"token2" + 0.167*"token5" + 0.167*"token3" + 0.167*"token4"
Topic: 6 
Words: 0.167*"token3" + 0.167*"token2" + 0.167*"token1" + 0.167*"token6" + 0.167*"token4" + 0.167*"token5"
Topic: 7 
Words: 0.306*"token5" + 0.306*"token4" + 0.306*"token6" + 0.028*"token2" + 0.028*"token3" + 0.028*"token1"
Topic: 8 
Words: 0.167*"token6" + 0.167*"token3" + 0.167*"token2

## With these wordclouds it lets me identify the keywords in a text where the size of the words represents their frequency. With this, i'll get a good idea of what a text is about before even reading it but without it being pre processed, therefore we can see how important it is to remove stop words etc. otherwise we get "misleading" data, because the word "og" is being used alot, but doesnt say anything about the text or helps us understand the text

### they are saved in the folder strucutre and are called: kmd.png & stylecloud.png

In [2]:
stylecloud.gen_stylecloud(file_path='kmd.txt',icon_name= "fas fa-apple-alt")

In [105]:
stylecloud.gen_stylecloud(file_path='kmdAftale.txt',
                          icon_name='fas fa-apple-alt',
                          colors='white',
                          background_color='black',
                          output_name='kmdAftale.png',
                          collocations=False)